In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.utils import class_weight
import numpy as np

# Settings
IMG_SIZE = 64
EPOCHS = 20
DATASET_PATH = "/content/drive/MyDrive/utkfaces/UTKFace"  # change if your dataset folder is elsewhere


In [ ]:
# Load and preprocess data
data, labels = [], []
print("Loading images...")

for file in os.listdir(DATASET_PATH):
    if file.endswith(".jpg"):
        try:
            race = int(file.split('_')[2])
            img_path = os.path.join(DATASET_PATH, file)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            data.append(img)
            labels.append(race)
        except:
            continue

In [ ]:
# Print race distribution
unique, counts = np.unique(labels, return_counts=True)
print("Race distribution:", dict(zip(unique, counts)))

X = np.array(data) / 255.0
y = np.array(labels)

In [ ]:
# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Define model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dense(5, activation='softmax')  # 5 race categories
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)

class_weights = dict(enumerate(weights))

# Train with weights to balance the distributions
model.fit(X_train, y_train, epochs=EPOCHS, class_weight=class_weights)


Loading images...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 79s 2s/step - accuracy: 0.2921 - loss: 4.8002
Epoch 2/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 79s 2s/step - accuracy: 0.2510 - loss: 1.6495
Epoch 3/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 81s 2s/step - accuracy: 0.5120 - loss: 1.5676
Epoch 4/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 81s 2s/step - accuracy: 0.3659 - loss: 1.3720
Epoch 5/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 73s 2s/step - accuracy: 0.6053 - loss: 0.9327
Epoch 6/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 79s 2s/step - accuracy: 0.7535 - loss: 0.5889
Epoch 7/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 78s 2s/step - accuracy: 0.8107 - loss: 0.4336
Epoch 8/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 75s 2s/step - accuracy: 0.8504 - loss: 0.3063
Epoch 9/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 81s 2s/step - accuracy: 0.8675 - loss: 0.2582
Epoch 10/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 82s 2s/step - accuracy: 0.9231 - loss: 0.1370
Epoch 11/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 74s 2s/step - accuracy: 0.9398 - loss: 0.1195
Epoch 12/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 82s 2s/step - accuracy: 0.9217 - lo

In [ ]:
model.save("/content/drive/MyDrive/ethnicity_model_2.h5")
print("✅ Model saved to Google Drive.")

✅ Model saved to Google Drive.
